In [1]:
import pandas as pd
import requests

In [5]:
# Create SQL Query
name_sensors = {
    #'D29ESP32DED36FA': 'Tangara_36FA',
    #'D29ESP32DED1CE2': 'Tangara_1CE2',
    #'D29ESP32DED1FCA': 'Tangara_1FCA',
    #'D29ESP32DED14D6': 'Tangara_14D6',
    #'D29ESP32DED2FF6': 'Tangara_2FF6',
    #'D29ESP32DED2492': 'Tangara_2492',
    #'D29TTGOT7D4D7A': 'Tangara_4D7A',
    'D29TTGOT7D48C6': 'CanAirIO_48C6',
    'D29TTGOT7D532E': 'CanAirIO_532E'
}

#condition = ""
#for key, value in name_sensors.items():
#    condition += f"\"name\" = '{key}' or "

#sql_query = "SELECT \"name\", last(\"pm25\") "\
#            "FROM fixed_stations_01 "\
#            f"WHERE ({condition[:-3]})"\
#                "and (time >= now() - 1h and time <= now()) "\
#                "GROUP BY time(30s) fill(none);"

sql_query = ""
for key, value in name_sensors.items():
    sql_query += "SELECT \"name\", last(\"pm25\") "\
                "FROM \"fixed_stations_01\" WHERE "\
                f"(\"name\" = '{key}') AND "\
                    "time >= now() - 1h and time <= now() "\
                "GROUP BY time(30s) fill(none); "

sql_query = sql_query[:-2]
sql_query

'SELECT "name", last("pm25") FROM "fixed_stations_01" WHERE ("name" = \'D29TTGOT7D48C6\') AND time >= now() - 1h and time <= now() GROUP BY time(30s) fill(none); SELECT "name", last("pm25") FROM "fixed_stations_01" WHERE ("name" = \'D29TTGOT7D532E\') AND time >= now() - 1h and time <= now() GROUP BY time(30s) fill(none)'

In [6]:
# Request to InfluxDB API REST

endpoint = "http://influxdb.canair.io:8086/query"
#endpoint = 'http://influxdb.canair.io:8000/api/datasources/proxy/1/query'
database = "canairio"

#headers = {
#    'Accept': 'application/csv',
#    'Content-Type': 'application/csv'
#}

#parameters = {
#    'q': sql_query,
#    'db': database
#}

parameters = {
    'db': database,
    'q': sql_query,
    #'epoch': 'ms'
}

#response = requests.get(endpoint, headers=headers, params=parameters)
response = requests.get(endpoint, params=parameters)

#print(response.text)
#response.json()

In [7]:
# Load Data Frame from JSON response
df_sensors_influxdb = pd.json_normalize(response.json()['results'], record_path=['series', 'values']).sort_values(by=[0])

# Create Data Frame from JSON
def create_df_sensors(df):
    df_sensors = []
    for key, value in name_sensors.items():
        df_sensor = df.loc[df[1] == key].reset_index(drop=True)[[0, 2]] # Warning
        df_sensor.rename(columns={0: 'Datetime', 2: name_sensors[key]}, inplace=True)
        df_sensor.set_index('Datetime', inplace=True)
        df_sensors.append(df_sensor)
    return df_sensors

df_sensors = create_df_sensors(df_sensors_influxdb)
df_sensors_influxdb = df_sensors[0].join(df_sensors[1:]).reset_index()
df_sensors_influxdb.head()

,Datetime,CanAirIO_48C6,CanAirIO_532E
0,2022-05-31T15:46:30Z,19.0,23.0
1,2022-05-31T15:48:00Z,23.0,23.0
2,2022-05-31T15:48:30Z,23.0,24.0
3,2022-05-31T15:49:00Z,24.0,23.0
4,2022-05-31T15:49:30Z,19.0,20.0
